# 1. Import and Install Dependencies

In [ ]:
!pip install opencv-python mediapipe scikit-learn matplotlib

import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard 

from PIL import ImageFont, ImageDraw, Image

In [ ]:
def mediapipe_detection(image, model): 
    #image = feed frame
    #model = Holistic model
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  
    results = model.process(image)                 
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [ ]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
def extract_keypoints(results):
    if results.left_hand_landmarks: #left hand landmarks
        lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() 
    else:
        lh = np.zeros(63) #21*3

    if results.right_hand_landmarks: #right hand landmarks
        rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() 
    else:
        rh = np.zeros(63) #21*3
    
    return np.concatenate([lh, rh])

In [ ]:
def putKoreanText(src, text, pos, font_size, font_color):
    img_pil = Image.fromarray(src)
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype('C:/Users/User/ActionDetectionforSignLanguage/fonts/gulim.ttc', font_size)
    draw.text(pos, text, font=font, fill= font_color)
    return np.array(img_pil)


In [ ]:
global mp_holistic
mp_holistic = mp.solutions.holistic 
global mp_drawing
mp_drawing = mp.solutions.drawing_utils 

global no_sequences
no_sequences= 60
global sequence_length
sequence_length = 60

global actions
actions = np.array(['house', 'thief', 'outofbreath', 'down', 'fire', 'stranger', 'car_accident'])
global actions_korean
actions_korean = ['집', '도둑', '숨이 안쉬어져요', '아래', '불이 났어요', '낯선사람', '교통사고']
global label_map
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
def main():
    model = Sequential()
    
    model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(60,126)))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, return_sequences=False, activation='relu'))

    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(actions.shape[0], activation='softmax'))
    
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    
    model.load_weights('kslaction_no_pose.h5')
    
    sequence = []
    sentence = []
    threshold = 0.7

    cap = cv2.VideoCapture(0)

    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            
            ret, frame = cap.read()
            image, results = mediapipe_detection(frame, holistic)
            print(results)
            draw_styled_landmarks(image, results)

            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-60:]

            if len(sequence) == 60:
                res = model.predict(np.expand_dims(sequence, axis=0))[0] 
                print(actions[np.argmax(res)])

                if res[np.argmax(res)] > threshold: 
                    cur_action_korean = actions_korean[np.argmax(res)]

                    if len(sentence) > 0: 
                        if cur_action_korean != sentence[-1]:
                            sentence.append(cur_action_korean)
                    else: 
                        sentence.append(cur_action_korean)

                if len(sentence) > 5: 
                    sentence = sentence[-5:]

                cv2.rectangle(image, (0,0), (640, 40), (0, 0, 0), -1) 
                image = putKoreanText(image, ' '.join(sentence), (3,10), 20, (255, 255, 255))

                cv2.imshow('OpenCV Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

In [ ]:
if __name__ == '__main__':
    print('KOKO.exe running...')
    main()